# Random Forest Regression

**Random Forest Regression** is an ensemble learning method for regression that operates by constructing a multitude of decision trees at training time and outputting the mean prediction of the individual trees. Random Forest combines multiple decision trees to increase the overall accuracy and reduce the risk of overfitting.

## Concept

Random Forest builds multiple decision trees and merges them together to get a more accurate and stable prediction. The ensemble of trees helps to correct for trees that may have overfitted to their sample. It is one of the most powerful and widely used machine learning algorithms due to its simplicity and diversity (applicable to both classification and regression tasks).

## How It Works

1. **Bootstrap Aggregating (Bagging):** Random Forest uses the bagging technique where it creates multiple datasets from the original dataset by sampling with replacement, and then trains a decision tree on each.
2. **Node Splitting:** At each node in the trees, a random subset of the features is considered for splitting. This introduces diversity in the model, reducing the variance and preventing overfitting.
3. **Averaging:** For regression tasks, it averages the outputs of different trees to give a final prediction.

## Key Parameters

- $n_{\text{estimators}}$: The number of trees in the forest. Typically, the more trees, the better to learn the data. However, adding a lot of trees can slow down the training process considerably.
- $\text{max\_depth}$: The maximum depth of the trees. Deeper trees learn more detailed data patterns and can be more prone to overfitting.
- $\text{min\_samples\_split}$: The minimum number of samples required to split an internal node.
- $\text{min\_samples\_leaf}$: The minimum number of samples required to be at a leaf node.

## Advantages

- Effective in high dimensional spaces and large data sets.
- Handles both numerical and categorical data.
- Provides a good indicator of the importance of features.

## Applications

Random Forest is versatile and can be used for:
- Predicting housing prices,
- Stock market behavior analysis,
- Medical diagnosis,
- Complex regression tasks in ecological and biological studies.

# Implementation

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from IPython.display import display, clear_output, HTML

import warnings
warnings.filterwarnings("ignore")

### Import and show Data

In [2]:
data = pd.read_csv('./Data/BeadArea.csv')
display(data.head())
print ("The data is composed of ", data.shape[0], " rows and ", data.shape[1], " columns.")

,Time,Position Feedback,Velocity Feedback from Axis 1,Velocity Feedback from Axis 2,Temperature,Current Feedback,Interpolated Bead Area
0,0.056,15.567,11.178,6.4952,1.1173,1.2309,1.1184
1,0.057,15.579,11.257,6.4952,1.1175,1.2735,1.1191
2,0.058,15.590,11.375,3.2476,1.1175,1.1567,1.1198
3,0.059,15.601,11.324,6.4952,1.1173,1.2975,1.1205
4,0.060,15.613,11.285,6.4952,1.1173,1.2168,1.1212


The data is composed of  295568  rows and  7  columns.


### Data Preprocessing

In [3]:
data['Lag1'] = data['Interpolated Bead Area'].shift(1)
data['Lag2'] = data['Interpolated Bead Area'].shift(2)
data['Lag3'] = data['Interpolated Bead Area'].shift(3)
data.dropna(inplace=True)

### Predict Bead Area

In [4]:


# Define widgets with adjusted layout
index_range_slider = widgets.IntRangeSlider(
    value=[0, min(500, len(data))],
    min=0,
    max=len(data),
    step=1,
    description='Index Range:',
    layout=widgets.Layout(width='600px'),  # Increase width for better readability
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

feature_select = widgets.SelectMultiple(
    options=['Lag1', 'Lag2', 'Lag3', 'Time', 'Position Feedback', 'Velocity Feedback from Axis 1', 'Velocity Feedback from Axis 2', 'Temperature', 'Current Feedback'],
    value=['Lag1', 'Lag2', 'Lag3', 'Time', 'Position Feedback', 'Velocity Feedback from Axis 1', 'Velocity Feedback from Axis 2', 'Temperature', 'Current Feedback'],
    description='Features:',
    layout=widgets.Layout(width='600px', height='180px'),  # Increase width and height
    style={'description_width': '150px'},  # Increase description width
    disabled=False
)

train_size_slider = widgets.IntSlider(
    value=80,
    min=50,
    max=95,
    step=1,
    description='Train %:',
    layout=widgets.Layout(width='600px'),  # Increase width
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

# Random Forest parameter sliders
n_estimators_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='Number of Estimators:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '200px'},
    continuous_update=False
)

max_depth_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description='Maximum Depth:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '200px'},
    continuous_update=False
)

min_samples_split_slider = widgets.IntSlider(
    value=2,
    min=2,
    max=20,
    step=1,
    description='Minimum Split Samples:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '200px'},
    continuous_update=False
)

apply_button = widgets.Button(description="Apply Changes", layout=widgets.Layout(width='800px'))

# Define the function to apply changes and update the plots
def apply_changes(b):
    with output:
        clear_output(wait=True)
        
        # Extract the parameters from widgets
        index_range = index_range_slider.value
        selected_features = list(feature_select.value)
        train_size_pct = train_size_slider.value / 100
        n_estimators = n_estimators_slider.value
        max_depth = max_depth_slider.value
        min_samples_split = min_samples_split_slider.value
        
        # Slice the data
        df = data[index_range[0]:index_range[1]]
        
        # Prepare the data (assuming 'Interpolated Bead Area' is already in `df`)
        X = df[selected_features]
        y = df['Interpolated Bead Area']
        
        # Train-test split
        train_size = int(len(df) * train_size_pct)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Train the model
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            random_state=42
        )
        model.fit(X_train, y_train)
        
        # Predict on test data
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        display(HTML(f'<b>Mean Squared Error: {mse:.5f}</b>'))  # Display MSE in bold
        
        # Plot predicted vs actual
        plt.figure(figsize=(10, 6))
        plt.plot(y_train.index, y_train, label='Training', color='green')
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.xlabel('Time')
        plt.ylabel('Interpolated Bead Area')
        plt.title('Actual vs Predicted Interpolated Bead Area')
        plt.legend()
        plt.show()
        
        # Calculate loss for each point
        pointwise_mse_loss = (y_test - y_pred) ** 2
        
        # Plot the pointwise loss
        plt.figure(figsize=(10, 6))
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.plot(y_test.index, pointwise_mse_loss, label='Pointwise MSE Loss', color='orange')
        plt.xlabel('Time')
        plt.ylabel('MSE Loss')
        plt.title('Pointwise MSE Loss of Predicted vs Actual Interpolated Bead Area')
        plt.legend()
        plt.show()

# Link the apply button to the function
apply_button.on_click(apply_changes)

# Display the widgets and the output area
output = widgets.Output()

display(index_range_slider, feature_select, train_size_slider, n_estimators_slider, max_depth_slider, min_samples_split_slider, apply_button, output)


IntRangeSlider(value=(0, 500), continuous_update=False, description='Index Range:', layout=Layout(width='600px…

SelectMultiple(description='Features:', index=(0, 1, 2, 3, 4, 5, 6, 7, 8), layout=Layout(height='180px', width…

IntSlider(value=80, continuous_update=False, description='Train %:', layout=Layout(width='600px'), max=95, min…

IntSlider(value=100, continuous_update=False, description='Number of Estimators:', layout=Layout(width='600px'…

IntSlider(value=10, continuous_update=False, description='Maximum Depth:', layout=Layout(width='600px'), max=5…

IntSlider(value=2, continuous_update=False, description='Minimum Split Samples:', layout=Layout(width='600px')…

Button(description='Apply Changes', layout=Layout(width='800px'), style=ButtonStyle())

Output()